# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv












2023-05-01	Learning Group Importance using the Differentiable Hypergeometric Distribution	ICLR 2023 (Spotlight Presentation)	We propose a differentiable formulation for the multivariate hypergeometric distribution.	"TM Sutter, L Manduchi, A Ryser, JE Vogt. (2023). ""Learning Group Importance using the Differentiable Hypergeometric Distribution"" <i>ICLR 2023</i>."	Learning Group Importance using the Differentiable Hypergeometric Distribution	https://arxiv.org/abs/2203.01629

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url
0,2020-06-05,Generation of differentially private heterogen...,Arxiv,Generating differentially private medical reco...,"K Chin-Cheong, TM Sutter, JE Vogt. (2020). ""Ge...",Generation of differentially private heterogen...,https://arxiv.org/abs/2006.03423
1,2020-09-28,Self-supervised disentanglement of modality-sp...,GCPR 2020,Disentangling shared and modality-specific lat...,"I Daunhawer, TM Sutter, R Marcinkevics, JE Vog...",Self-supervised disentanglement of modality-sp...,https://link.springer.com/chapter/10.1007/978-...
2,2020-12-18,A comparison of general and disease-specific m...,Journal of the American Medical Informatics As...,Comparing different models with respect to the...,"TM Sutter, JA Roth, K Chin-Cheong, BL Hug, JE ...",A comparison of general and disease-specific m...,https://academic.oup.com/jamia/article-abstrac...
3,2020-12-05,Multimodal Generative Learning Utilizing the J...,Neurips 2020,Utilizing the Jensen-Shannon divergence in a V...,"TM Sutter, I Daunhawer, JE Vogt. (2020). ""Mult...",Multimodal Generative Learning Utilizing the J...,https://proceedings.neurips.cc/paper/2020/hash...
4,2021-01-08,Machine Learning Algorithms Evaluate Immune Re...,Frontiers in Cellular and Infection Microbiology,Identifying optimal antigen-cytokine combinati...,"NR Meier, TM Sutter, M Jacobsen, THM Ottenhoff...",Machine Learning Algorithms Evaluate Immune Re...,https://www.frontiersin.org/articles/10.3389/f...
5,2021-04-26,Generalized Multimodal ELBO,ICLR 2021,Generalizing previous formulations of scalable...,"TM Sutter, I Daunhawer, JE Vogt. (2021). ""Gene...",Generalized Multimodal ELBO,https://arxiv.org/abs/2110.04121
6,2021-07-07,Multimodal Generative Learning on the Mimic-CX...,MIDL 2021 Short Paper,Evaluating MoPoE-VAE on the Mimic-Cxr database.,"HJ Klug, TM Sutter, JE Vogt. (2021). ""Multimod...",Multimodal Generative Learning on the Mimic-CX...,https://openreview.net/pdf?id=ZVqjoKVbYMl
7,2021-12-13,Multimodal Relational VAE,Neurips 2021 Workshop on Bayesian Deep Learning,Explicitely modelling and learning the relatio...,"TM Sutter, JE Vogt. (2021). ""Multimodal Relat...",Multimodal Relational VAE,https://www.research-collection.ethz.ch/bitstr...
8,2022-04-25,On the Limitations of Multimodal VAEs,ICLR 2022,"We show limitations of current scalable, multi...","I Daunhawer, TM Suttter, K Chin-Cheong, E Palu...",On the Limitations of Multimodal VAEs,https://arxiv.org/abs/2110.04121
9,2022-12-02,M(otion)-mode Based Prediction of Cardiac Func...,Medical Imaging meets NeurIPS Workshop 2022,Artificial (M)otion-mode based predicting of l...,"TM Sutter, S Balzer, E Ozkan, JE Vogt. (2022) ...",M(otion)-mode Based Prediction of Cardiac Func...,https://www.research-collection.ethz.ch/handle...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [5]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + str(item.url_slug) + ".md"
    html_filename = str(item.pub_date) + "-" + str(item.url_slug)
    year = str(item.pub_date)[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + str(item.title) + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + str(html_filename)
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(str(item.excerpt)) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(str(item.venue)) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(str(item.citation)) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + str(item.citation)
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [6]:
!ls ../_publications/

2020-06-05-Generation of differentially private heterogeneous electronic health records .md
2020-09-28-Self-supervised disentanglement of modality-specific and shared factors improves multimodal generative model.md
2020-12-05-Multimodal Generative Learning Utilizing the Jensen-Shannon Divergence.md
2020-12-18-A comparison of general and disease-specific machine learning models for the prediction of unplanned hospital readmissions .md
2021-01-08-Machine Learning Algorithms Evaluate Immune Response to Novel Mycobacterium tuberculosis Antigens for Diagnosis of Tuberculosis.md
2021-04-26-Generalized Multimodal ELBO.md
2021-07-07-Multimodal Generative Learning on the Mimic-CXR database .md
2021-12-13-Multimodal Relational VAE.md
2022-04-25-On the Limitations of Multimodal VAEs.md
2022-12-02-M(otion)-mode Based Prediction of Cardiac Function on Echocardiograms.md
2023-05-01-Learning Group Importance using the Differentiable Hypergeometric Distribution.md


In [7]:
!cat "../_publications/2023-05-01-Learning Group Importance using the Differentiable Hypergeometric Distribution.md"

---
title: "Learning Group Importance using the Differentiable Hypergeometric Distribution"
collection: publications
permalink: /publication/2023-05-01-Learning Group Importance using the Differentiable Hypergeometric Distribution
excerpt: 'We propose a differentiable formulation for the multivariate hypergeometric distribution.'
date: 2023-05-01
venue: 'ICLR 2023 (Spotlight Presentation)'
paperurl: 'https://arxiv.org/abs/2203.01629'
citation: 'TM Sutter, L Manduchi, A Ryser, JE Vogt. (2023). &quot;Learning Group Importance using the Differentiable Hypergeometric Distribution&quot; <i>ICLR 2023</i>.'
---
We propose a differentiable formulation for the multivariate hypergeometric distribution.

[Download paper here](https://arxiv.org/abs/2203.01629)

Recommended citation: TM Sutter, L Manduchi, A Ryser, JE Vogt. (2023). "Learning Group Importance using the Differentiable Hypergeometric Distribution" <i>ICLR 2023</i>.